# LGB_1 creation

Most of this is set up so that multiple models can be run. All of the
calculations are looped over model names. Each relevant dataframe is loaded
into a dictionary, accessed by model name. We run it just for
the model lgb_1

## Packages & Preamble

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
import optuna
from sklearn.preprocessing import OrdinalEncoder
from return_utils import pos_to_return,collecting_stock_data_avg,collecting_stock_percentages, factors_to_pos
import shap
import empyrical

import matplotlib.pyplot as plt
from Stock_tda_calc import s_and_p


/usr/local/Caskroom/miniconda/base/envs/tdasp500/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/Caskroom/miniconda/base/envs/tdasp500/lib/python3.9/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
%load_ext autoreload
%autoreload 2

Load the list of tickers: note that we remove all appearing post 2017 to avoid
selection bias.

Collect stock close and returns

In [3]:
stock_closes_df = collecting_stock_data_avg(tickers=s_and_p)
stock_closes_df.index=pd.to_datetime(stock_closes_df.index.values,format='%Y-%m-%d')
stock_volumes_df=collecting_stock_data_avg(tickers=s_and_p,value_name='Volume').mean(axis=1)
stock_volumes_df.index=pd.to_datetime(stock_closes_df.index.values,format='%Y-%m-%d')
stock_perc_df = collecting_stock_percentages(s_and_p)
stock_perc_df.index=pd.to_datetime(stock_perc_df.index.values,format='%Y-%m-%d')
stock_closes_rets_df=pd.concat([stock_closes_df,stock_perc_df],axis=1)

Collect bnh returns only

In [4]:
bnh_returns_df=stock_perc_df.mean(axis=1)

# MACHINE LEARNING STRATEGY

## Collect data

These are the topological factors

In [5]:
betti_values=pd.read_csv('Betti_experiments/Sharpe_values/betti_dim_3_points_5/relative_betti_values.csv',index_col='Date')
long_betti_values=pd.read_csv('Betti_experiments/Sharpe_values/betti_dim_3_points_9/relative_betti_values.csv',index_col='Date')
bmd_factors=pd.read_csv('Betti_experiments/Sharpe_values/betti_dim_3_points_5/relative_meandiff_factors.csv',index_col='Date')
bmd_pos=factors_to_pos(stock_factor_df=bmd_factors,collected_stocks_df=stock_perc_df,tickers=s_and_p)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Run the below to make sure everything has datetime indices

In [6]:
bnh_returns_df.index=pd.to_datetime(bnh_returns_df.index.values,format='%Y-%m-%d')
for df in [long_betti_values,betti_values,bmd_factors]:
    df.index=pd.to_datetime(df.index.values,format='%Y-%m-%d')

The below function calculates wheter a stock is 'high' or 'low' in some
attribute, in reference to 'compar_ser'

In [7]:
def high_value(input_ser,compar_ser,output_ser):
    ind=compar_ser.index.intersection(input_ser.index)
    df=output_ser.loc[ind]
    input_ser=input_ser.loc[ind]
    compar_ser=compar_ser.loc[ind]
    (df
    .loc[
        input_ser.rolling(window=20).mean()
        >
        compar_ser.rolling(window=20).mean()
    ]       
    )=1
    (df
        .loc[input_ser.rolling(window=20).mean()
            <=
            compar_ser.rolling(window=20).mean()
            ]       
    )=0

    df.loc[df.isnull()&(input_ser>compar_ser)]=1
    df.loc[df.isnull()&(input_ser<compar_ser)]=0
    return df.shift(1)



Set indexers, and call a few dataframes

In [8]:
future_window=5
long_fut=10
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=future_window)
long_indexer=pd.api.indexers.FixedForwardWindowIndexer(window_size=long_fut)
bnh_returns_df.index=pd.to_datetime(bnh_returns_df.index.values,format='%Y-%m-%d')
bns=bnh_returns_df.rolling(window=future_window).sum()
bnm=bnh_returns_df.rolling(window=future_window).mean()
bnstd=bnh_returns_df.rolling(window=future_window).std()
mean_stocks=stock_closes_df.mean(axis=1)
mean_stocks.index=pd.to_datetime(mean_stocks.index.values,'%Y-%m-%d')




Loop over stocks and calculate various attributes

In [9]:

for t in tqdm(s_and_p):
    df=pd.read_csv(f'Persim_vs_Betti/landscape/{t}.csv',index_col=0)
    df.index=pd.to_datetime(df.index.values,format='%Y-%m-%d')

    df.loc[:,'High value']=pd.to_numeric(high_value(df.loc[:,'Adj Close'],mean_stocks,df.loc[:,'Adj Close']))
    df.loc[:,'High volume']=pd.to_numeric(high_value(df.loc[:,'Volume'],stock_volumes_df,df.loc[:,'Volume']))
    df['Price momentum']=df['Adj Close'].rolling(window=20).mean()/df['Adj Close'].rolling(window=60).mean()
    df['Landscape values']=df['Persistence landscape norm']

    df=df.loc[:,['Price momentum', 'Landscape values','High value','High volume']]

    df['ticker']=t
    df['Returns']=stock_perc_df[t +' % Return']
    df['Betti values dim=3 points=5']=betti_values[t]
    df['Betti values dim=3 points=9']=long_betti_values[t]

    df['Relative return']=(df['Returns']-bnh_returns_df)
    df['Relative return momentum']=(df['Relative return'].rolling(window=10).mean()/df['Relative return'].rolling(window=30).mean())
    df['Return momentum']=(df['Returns'].rolling(window=10).mean()/df['Returns'].rolling(window=30).mean())
    df['Betti momentum']=(df['Betti values dim=3 points=5']/df['Betti values dim=3 points=9'])
    df['Betti meandiff']=bmd_factors[t]
    # df['Betti meandiff strategy position']=bmd_pos[f'{t} Position']
    df['Landscape mean diff']=(df['Landscape values']-df['Landscape values'].rolling(window=21).mean())
    df[f'{long_fut} day past relative return']=df[f'Relative return'].rolling(window=long_fut).mean()
    df[f'{future_window} day past relative return']=df[f'Relative return'].rolling(window=future_window).mean()
    df[f'{long_fut} day future relative return']=df[f'Relative return'].shift(-1).rolling(window=long_indexer).mean()
    df[f'{future_window} day future relative return']=df[f'Relative return'].shift(-1).rolling(window=indexer).mean()
    df['1 day future relative return']=df[f'Relative return'].shift(-1)
    df.index.name='Date'
    df.to_csv('ML_experiments/ML_dataframes/'+t+'.csv')


100%|██████████| 407/407 [00:38<00:00, 10.48it/s]


Create test etc data

In [10]:
fit_data=pd.DataFrame(dtype=np.float64)
valid_data=pd.DataFrame(dtype=np.float64)
test_data=pd.DataFrame(dtype=np.float64)
train_data=pd.DataFrame(dtype=np.float64)

Below sets start/end dates for fit, validation, training (=fit + validation),
and test

In [11]:
dates={'fit_start':'2017-08-11','fit_end':'2020-06-30', 'valid_start':'2020-07-01',
'valid_end':'2020-12-31', 'train_start':'2017-08-11',
'train_end':'2020-12-31','test_start':'2021-01-01', 'test_end': '2022-06-01'}
d_splits=['fit','valid','train','test']
data_split_dfs={'fit':fit_data,'valid':valid_data,'train':train_data,'test':test_data}

### Run below first time only

Create fit/validation/training/test data

In [ ]:
for sp in d_splits:
        data_split_dfs[sp]=pd.DataFrame(dtype=np.float64)
        for t in s_and_p:
            df=pd.read_csv('ML_experiments/ML_dataframes/'+t+'.csv',index_col='Date')
            df.index=pd.to_datetime(df.index.values,format='%Y-%m-%d')
            df.index=df.index.rename('Date')
            df=df.loc[dates[sp+'_start']:dates[sp+'_end']]
            data_split_dfs[sp]=pd.concat([data_split_dfs[sp],df])
        data_split_dfs[sp].to_csv('ML_experiments/ML_'+sp+'.csv')

### Run second time

In [13]:
fit_data=pd.read_csv('ML_experiments/ML_fit.csv', low_memory=False,index_col='Date')
valid_data=pd.read_csv('ML_experiments/ML_valid.csv', low_memory=False,index_col='Date')
test_data=pd.read_csv('ML_experiments/ML_test.csv', low_memory=False,index_col='Date')
train_data=pd.read_csv('ML_experiments/ML_train.csv',low_memory=False,index_col='Date')
for df in [fit_data,valid_data,test_data]:
        df.index=pd.to_datetime(df.index.values,format='%Y-%m-%d')

### Finish collecting data

We use the below in model construction

In [15]:
drops=['ticker','1 day future relative return',
        f'{future_window} day future relative return',f'{long_fut} day future relative return']
X_fit=fit_data.copy()
X_fit.index.name='Date'
X_test=test_data.copy()
X_test.index.name='Date'
X_valid=valid_data.copy()
X_valid.index.name='Date'
X_train=train_data.copy()
X_train.index.name='Date'
for df in [X_fit,X_valid,X_test,X_train]:
    df.drop(drops, axis=1,inplace=True)

Create target variables. Note that we keep model names and use dictionaries to
define target variables, models, etc, so can add/remove models easily

In [16]:
y_next_fit = fit_data.copy()[['1 day future relative return']]
y_next_valid = valid_data.copy()[['1 day future relative return']]
y_next_test = test_data.copy()[['1 day future relative return']]
y_next_train = train_data.copy()[['1 day future relative return']]

model_names=['lgb_1']

predict_data={'lgb_1':[y_next_fit,y_next_valid,y_next_train,y_next_test]      
        }
time_scale={'lgb_1':1}

for mod in model_names:
    for i in range(4):
        predict_data[mod][i].index.name='Date'
        predict_data[mod][i]=( predict_data[mod][i][f'{time_scale[mod]} day future relative return']> 0)*1
      
             



## Optimize model params

Use Optuna to optimize model parameters

In [17]:
def objective(trial, modeller,X, y,Xv,yv):
    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=10),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 10000, step=50),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=1),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=1),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.05
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.05
        ),
    }
    model = modeller()
    model.fit(
            X,
            y
    )
    # preds = model.predict(Xv)
    return (model.score(Xv,yv))

In [18]:
lgb_1_ret = lgb.LGBMClassifier()

model_name_dict={
    'lgb_1':lgb_1_ret
                }

In [32]:
optuna.logging.set_verbosity(0)
best_param_dict={}
for mod in model_names:
    study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
    Xm=X_fit.reset_index().drop('Date',axis=1)
    ym=predict_data[mod][0].reset_index().drop('Date',axis=1)
    Xmv=X_valid.reset_index().drop('Date',axis=1)
    ymv=predict_data[mod][1].reset_index().drop('Date',axis=1)

    func = lambda trial: objective(trial, lgb.LGBMClassifier,Xm,ym.iloc[:,0].ravel(),Xmv,ymv.iloc[:,0].ravel())
    study.optimize(func, n_trials=20)
    best_param_dict[mod]=pd.DataFrame(study.best_params.items(),columns=['params','value'])
    model_name_dict[mod]=lgb.LGBMClassifier(**study.best_params)
    model_name_dict[mod].fit(X_train.reset_index().drop('Date',axis=1),predict_data[mod][2].reset_index().drop('Date',axis=1).iloc[:,0].ravel())
    print(f'{mod} fitted')
#model_name_dict[mod].booster_.save_model('ML_experiments/models/'+mod+'_lgb_model.txt')


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=350, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=350
[LightGBM] [Warning] lambda_l2 is set=86, reg_lambda=0.0 will be ignored. Current value: lambda_l2=86
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] min_gain_to_split is set=10.804999368148568, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=10.804999368148568
[LightGBM] [Warning] lambda_l1 is set=78, reg_alpha=0.0 will be ignored. Current value: lambda_l1=78
lgb_1 fitted


## Predict and evaluate

In [20]:
test_data=test_data.loc[dates['test_start']:dates['test_end']]

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.


In [33]:
rets_dict={}
pos_dict={}
performance_stats_ml=pd.DataFrame(dtype=np.float64)
for mod in model_names:
    final_pos=stock_closes_rets_df.copy().loc[dates['test_start']:dates['test_end']]
    pos=pd.DataFrame(dtype=np.float64)
    for t in s_and_p:
        df=test_data.loc[test_data['ticker']==t]
        df=df.drop(drops,axis=1)
        y_pred=model_name_dict[mod].predict(df)
        y_pred=(
            pd.DataFrame(y_pred,index=df.index.values,columns=[t+' Position'])
            .shift(1)
        )
        pos=pd.concat([pos,y_pred],axis=1)
    final_pos=pd.concat([pos,final_pos],axis=1)
    pos_dict[mod]=final_pos
    print(f'{mod} positions calculated')
    rets=pos_to_return(final_pos, tickers=s_and_p, start=dates['test_start'], end=dates['test_end'],  hold_len=1)
    rets[rets.isna()] = 0
    rets_dict[mod]=rets
    print(f'{mod} returns calculated')
    perf_values=pd.DataFrame({
        'Sharpe':empyrical.sharpe_ratio(rets),
        'Sortino':empyrical.sortino_ratio(rets),
        'alpha':empyrical.alpha(rets,bnh_returns_df),
        'beta':empyrical.beta(rets,bnh_returns_df)
    },index=[f'{mod}'])
    performance_stats_ml=pd.concat([performance_stats_ml,perf_values])

lgb_1 positions calculated
lgb_1 returns calculated


In [34]:
bnh_perf=pd.DataFrame({
        'Sharpe':empyrical.sharpe_ratio(bnh_returns_df.loc[dates['test_start']: dates['test_end']]),
        'Sortino':empyrical.sortino_ratio(bnh_returns_df.loc[dates['test_start']: dates['test_end']]),
        'alpha':'-',
        'beta':'-'
    },index=['bnh'])
bnh_perf.to_csv('ML_experiments/Sharpe_vals/bnh_performance.csv')
performance_stats_for_csv=pd.concat([performance_stats_ml,bnh_perf])
performance_stats_for_csv.index.name='Strategy'

print(performance_stats_for_csv)

            Sharpe   Sortino    alpha      beta
Strategy                                       
lgb_1     1.050135  1.493223  0.02055  1.001086
bnh       0.952194  1.354368        -         -


In [39]:
print(performance_stats_for_csv.to_latex())

\begin{tabular}{lrrll}
\toprule
{} &    Sharpe &   Sortino &    alpha &      beta \\
Strategy &           &           &          &           \\
\midrule
lgb\_1    &  1.050135 &  1.493223 &  0.02055 &  1.001086 \\
bnh      &  0.952194 &  1.354368 &        - &         - \\
\bottomrule
\end{tabular}



In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.


## Shap values

In [27]:
X_shaps=X_train.sample(n=10000,random_state=31415)

In [ ]:
model=lgb.Booster(model_file=f'ML_experiments/Models/lgb_1_lgb_model.txt')
model.params['objective']='binary'
lgb_exp = shap.TreeExplainer(model,X_shaps)
shap_values=lgb_exp(X_shaps)
shap.summary_plot(shap_values,features=X_shaps,show=False,color_bar=False)
cbar=plt.colorbar(aspect=100)
cbar.set_ticks([-0.0175,0.0175])
cbar.set_ticklabels(['low value','high value'])
plt.show()

# Options to save model outcomes etc

In [35]:
save=False

In [36]:
if save==True:
    model_names=['lgb_1']
    for mod in model_names:
        best_param_dict[mod].to_csv(f'ML_experiments/Models/{mod}_params.csv')
        model_name_dict[mod].booster_.save_model(f'ML_experiments/Models/{mod}_model.txt')
        rets_dict[mod].to_csv(f'ML_experiments/Models/{mod}_returns.csv')
        pos_dict[mod].to_csv(f'ML_experiments/Models/{mod}_ret_pos.csv')
    performance_stats_for_csv.to_csv(f'ML_experiments/Sharpe_vals/{mod}_performance stats.csv')